# 自动求导

## autograd
用Tensor训练网络很方便,对于像线性回归等较为简单的模型来说,手动实现反向传播还可以应付，但实际使用中经常出现非常复杂的网络结构，此时如果手动实现反向传播，不仅费时费力，而且容易出错，难以检查。torch.autograd就是为方便用户使用，而专门开发的一套自动求导引擎，它能够根据输入和前向传播过程自动构建计算图，并执行反向传播。

计算图(Computation Graph)是现代深度学习框架如PyTorch和TensorFlow等的核心，其为高效自动求导算法——反向传播(Back Propogation)提供了理论支持，了解计算图在实际写程序过程中会有极大的帮助。

## 1 requires_grad

PyTorch在autograd模块中实现了计算图的相关功能，autograd中的核心数据结构是Variable。从v0.4版本起，Variable和Tensor合并。我们可以认为需要求导(requires_grad)的tensor即Variable。autograd记录对tensor的操作记录用来构建计算图。

Variable提供了大部分tensor支持的函数，但其不支持部分inplace函数，因这些函数会修改tensor自身，而在反向传播中，variable需要缓存原来的tensor来计算反向传播梯度。如果想要计算各个Variable的梯度，只需调用根节点variable的backward方法，autograd会自动沿着计算图反向传播，计算每一个叶子节点的梯度。

variable.backward(gradient=None, retain_graph=None, create_graph=None)主要有如下参数：

（1）grad_variables：形状与variable一致，对于y.backward()，grad_variables相当于链式法则 ${dz \over dx}={dz \over dy} \times {dy \over dx}$中的 $\textbf {dz} \over \textbf {dy}$。grad_variables也可以是tensor或序列。
（2）retain_graph：反向传播需要缓存一些中间结果，反向传播之后，这些缓存就被清空，可通过指定这个参数不清空缓存，用来多次反向传播。 

（3）create_graph：对反向传播过程再次构建计算图，可通过backward of backward实现求高阶导数。
上述描述可能比较抽象，如果没有看懂，不用着急，会在本节后半部分详细介绍，下面先看几个例子。

In [1]:
from __future__ import print_function
import torch as t

In [2]:
#在创建tensor是指定requires_grad
a = t.randn(3, 4, requires_grad = True)

#或者
a = t.randn(3, 4).requires_grad_()

#或者
a = t.randn(3, 4)
a.requires_grad = True
a

tensor([[-2.2155,  1.2681, -0.9808, -2.1088],
        [ 0.3566,  0.2245, -0.4443, -0.3617],
        [-1.1861,  0.5124, -0.4224, -0.1794]], requires_grad=True)

In [3]:
b = t.zeros(3, 4).requires_grad_()
b

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], requires_grad=True)

In [4]:
c = a.add(b)
#也可以写成
#c = a + b

c

tensor([[-2.2155,  1.2681, -0.9808, -2.1088],
        [ 0.3566,  0.2245, -0.4443, -0.3617],
        [-1.1861,  0.5124, -0.4224, -0.1794]], grad_fn=<ThAddBackward>)

In [5]:
d = c.sum()
d.backward()#反向传播

In [7]:
d
d.requires_grad #d此时还是一个requires_grad 为 True的tensor，对他的操作要慎重

True

In [8]:
a.grad#计算a的导数

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [10]:
print(a.requires_grad, b.requires_grad, c.requires_grad)  #虽然c没有设置需要求导，但c依赖于a，而a需要求导，故c的requires_grad = True

True True True


In [39]:
print(a.is_leaf, b.is_leaf, c.is_leaf)

True True False


In [40]:
#c.grad为None，因为c不是叶子节点，c的梯度是为了计算a的梯度，
#所以虽然c.requires_grad = True, 但梯度计算完后立即被释放
print(c.grad)

None


计算下面这个函数的导函数：
$$
y = x^2\bullet e^x
$$
它的导函数是：
$$
{dy \over dx} = 2x\bullet e^x + x^2 \bullet e^x
$$
来看看autograd的计算结果与手动求导计算结果的误差。

In [41]:
def f(x):
    """计算y = f(x)"""
    y = x ** 2 * t.exp(x)
    return y

In [42]:
def gradf(x):
    '''手动计算f(x)的导数'''
    grad_f = 2 * x * t.exp(x) + x ** 2 * t.exp(x)
    return grad_f

In [43]:
x = t.randn(3, 4, requires_grad = True)
y = f(x)
print(y)

tensor([[ 0.0072,  0.1211,  0.0416,  0.5191],
        [ 0.3448,  0.0013,  0.0062,  0.5243],
        [41.1863,  0.1310,  0.4001,  1.0945]], grad_fn=<ThMulBackward>)


In [44]:
y.backward(t.ones(3, 4)) #gradient 形状与y一致
x.grad

tensor([[ 0.1838, -0.4397, -0.3222, -0.1225],
        [-0.3895,  0.0761,  0.1696, -0.1061],
        [79.1582, -0.4458, -0.3328,  4.1043]])

In [45]:
#看看手动计算的导数
grad_x = gradf(x)
grad_x
#计算结果与autograd计算结果一致

tensor([[ 0.1838, -0.4397, -0.3222, -0.1225],
        [-0.3895,  0.0761,  0.1696, -0.1061],
        [79.1582, -0.4458, -0.3328,  4.1043]], grad_fn=<ThAddBackward>)

## 计算图

PyTorch中`autograd`的底层采用了计算图，计算图是一种特殊的有向无环图（DAG），用于记录算子与变量之间的关系。一般用矩形表示算子，椭圆形表示变量。如表达式$ \textbf {z = wx + b}$可分解为$\textbf{y = wx}$和$\textbf{z = y + b}$，其计算图如图3-3所示，图中`MUL`，`ADD`都是算子，$\textbf{w}$，$\textbf{x}$，$\textbf{b}$即变量。

![图3-3:computation graph](imgs/com_graph.svg)

如上有向无环图中，$\textbf{X}$和$\textbf{b}$是叶子节点（leaf node），这些节点通常由用户自己创建，不依赖于其他变量。$\textbf{z}$称为根节点，是计算图的最终目标。利用链式法则很容易求得各个叶子节点的梯度。
$${\partial z \over \partial b} = 1,\space {\partial z \over \partial y} = 1\\
{\partial y \over \partial w }= x,{\partial y \over \partial x}= w\\
{\partial z \over \partial x}= {\partial z \over \partial y} {\partial y \over \partial x}=1 * w\\
{\partial z \over \partial w}= {\partial z \over \partial y} {\partial y \over \partial w}=1 * x\\
$$
而有了计算图，上述链式求导即可利用计算图的反向传播自动完成，其过程如图3-4所示。

![图3-4：计算图的反向传播](imgs/com_graph_backward.svg)


在PyTorch实现中，autograd会随着用户的操作，记录生成当前variable的所有操作，并由此建立一个有向无环图。用户每进行一个操作，相应的计算图就会发生改变。更底层的实现中，图中记录了操作`Function`，每一个变量在图中的位置可通过其`grad_fn`属性在图中的位置推测得到。在反向传播过程中，autograd沿着这个图从当前变量（根节点$\textbf{z}$）溯源，可以利用链式求导法则计算所有叶子节点的梯度。每一个前向传播操作的函数都有与之对应的反向传播函数用来计算输入的各个variable的梯度，这些函数的函数名通常以`Backward`结尾。下面结合代码学习autograd的实现细节。

In [46]:
x = t.ones(1)
w = t.rand(1, requires_grad = True)
b = t.rand(1, requires_grad = True)
y = w * x
z = y + b

In [47]:
x.requires_grad, w.requires_grad, b.requires_grad

(False, True, True)

In [48]:
#虽然未指定y.requires_grad = True, 但y依赖于w，而w.requires_grad = True
#故 y.requires_grad = True
y.requires_grad

True

In [49]:
x.is_leaf, w.is_leaf, b.is_leaf

(True, True, True)

In [50]:
y.is_leaf, z.is_leaf

(False, False)

In [51]:
#grad_fn可以查看这个Variable的反向传播函数
#z是add函数的输出，故grad_fn为AddBackWard
z.grad_fn

In [52]:
#next_function保存grad_fn的输入，是一个tuple,tuple的元素也是Function
#第一个是y，它是乘法（mul）的输出，故grad_fn是MulBackWard
#第二个是b，它是用户创建的叶子节点，故b.grad_fn为None
z.grad_fn.next_functions

((<ThMulBackward at 0x1dba8f6bbe0>, 0), (<AccumulateGrad at 0x1dba8f6b978>, 0))

In [56]:
#Variable的grad_fn与图中function相对应
z.grad_fn.next_functions[0][0] == y.grad_fn

True

In [58]:
# 第一个是w，叶子节点，需要求导，梯度是累加的
# 第二个是x，叶子节点，不需要求导，所以为None
y.grad_fn.next_functions

((<AccumulateGrad at 0x1dba8f8f0f0>, 0), (None, 0))

In [59]:
#叶子节点的grad_fn是None
w.grad_fn, b.grad_fn, x.grad_fn

(None, None, None)

计算w的梯度的时候，需要用到x的数值(${\partial y\over \partial w} = x $)，这些数值在前向过程中会保存成buffer，在计算完梯度之后会自动清空。为了能够多次反向传播需要指定`retain_graph`来保留这些buffer。

In [60]:
#使用retain_graph来保存buffer
z.backward(retain_graph = True)
w.grad

tensor([1.])

In [61]:
#多次反向传播，梯度累加
z.backward(retain_graph = True)
w.grad

tensor([2.])

PyTorch使用的是动态图，它的计算图在每次前向传播时都是从头开始构建，所以它能够使用Python控制语句（如for、if等）根据需求创建计算图。这点在自然语言处理领域中很有用，它意味着你不需要事先构建所有可能用到的图的路径，图在运行时才构建。

In [63]:
def abs(x):
    if x>0:
        return x
    else:
        return - x
    
x = t.randn(1, requires_grad = True)
y = abs(x)
y.backward()
x.grad

tensor([-1.])

In [70]:
x = -1 * t.ones(1)
x.requires_grad = True
y = abs(x)
y.backward()
x.grad

tensor([-1.])

变量的`requires_grad`属性默认为False，如果某一个节点requires_grad被设置为True，那么所有依赖它的节点`requires_grad`都是True。这其实很好理解，对于$ \textbf{x}\to \textbf{y} \to \textbf{z}$，x.requires_grad = True，当需要计算$\partial z \over \partial x$时，根据链式法则，$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \frac{\partial y}{\partial x}$，自然也需要求$ \frac{\partial z}{\partial y}$，所以y.requires_grad会被自动标为True. 



有些时候我们可能不希望autograd对tensor求导。认为求导需要缓存许多中间结构，增加额外的内存/显存开销，那么我们可以关闭自动求导。对于不需要反向传播的情景（如inference，即测试推理时），关闭自动求导可实现一定程度的速度提升，并节省约一半显存，因其不需要分配空间计算梯度。

In [90]:
x = t.ones(1, requires_grad = True)
w = t.rand(1, requires_grad = True)
y = w * x
#y依赖于w和x，w.requires_grad = True, x.requires_grad = True
x.requires_grad , w.requires_grad, y.requires_grad

(True, True, True)

In [91]:
#只要y依赖的节点的requires_grad = True,y.requires_grad = True
# x = t.ones(1)
# w = t.rand(1, requires_grad = True)
x = t.ones(1, requires_grad = True)
w = t.rand(1)
y = w * x
y.requires_grad

True

In [93]:
with t.no_grad():
    x = t.ones(1)
    w = t.rand(1, requires_grad = True)
    y = w * x
# y依赖于w和x，虽然w.requires_grad = True，但是y的requires_grad依旧为False
y.requires_grad

False

t.no_grad()???
t.no_grad()应该是一种如果依赖的叶子节点中有一个requires_grad = False, 则根节点的requires_grad = True的方法，能够不计算导数,与下面的t.set_grad_enabled(False)有类似效果

In [101]:
t.set_grad_enabled(False)
x = t.ones(1)
w = t.rand(1, requires_grad = True)
y = w * x
y.requires_grad

False

In [102]:
#恢复默认配置
t.set_grad_enabled(True)
x = t.ones(1)
w = t.rand(1, requires_grad = True)
y = w * x
y.requires_grad

True

### 如果我们想修改tensor的值，又不想被autograd，可以对tensor.data进行操作

In [112]:
a = t.ones(3, 4, requires_grad = True)
b = t.randn(3, 4, requires_grad = True)
c = a * b

a.data #a.data还是一个tensor

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [113]:
a.data.requires_grad  #但是a.data已经独立于计算图之外

False

In [114]:
d = a.data.sigmoid_() #sigmoid_()是inplace操作，会修改a自身的值
d.requires_grad

False

In [115]:
a

tensor([[0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311]], requires_grad=True)

如果我们希望对tensor操作，但是又不希望被记录, 可以使用tensor.data 或者tensor.detach()

In [116]:
a.requires_grad

True

In [118]:
#类似于tensor = a.data, 但如果tensor被修改，backward可能会报错
tensor = a.detach()
tensor.requires_grad

False

In [121]:
#统计tensor的一些指标，不希望被记录
mean = tensor.mean()
std = tensor.std()
maximum = tensor.max()
print(mean, std, maximum)

tensor(0.7311) tensor(0.) tensor(0.7311)


In [124]:
tensor[0] = 1
# 下面会报错：　RuntimeError: one of the variables needed for gradient
#             computation has been modified by an inplace operation
#　因为 c=a*b, b的梯度取决于a，现在修改了tensor，其实也就是修改了a，梯度不再准确
#c.sum().backward()

在反向传播过程中非叶子节点的导数计算完之后即被清空。若想查看这些变量的梯度，有两种方法：
- 使用autograd.grad函数
- 使用hook

`autograd.grad`和`hook`方法都是很强大的工具，更详细的用法参考官方api文档，这里举例说明基础的使用。推荐使用`hook`方法，但是在实际使用中应尽量避免修改grad的值。

In [127]:
x = t.ones(3, requires_grad = True)
w = t.rand(3, requires_grad = True)
y = w * x
z = y.sum()
x.requires_grad, w.requires_grad, y.requires_grad, z.requires_grad

(True, True, True, True)

In [128]:
z.backward()
x.grad, w.grad, y.grad #非叶子节点grad计算完后自动清空，为None

(tensor([0.7573, 0.0519, 0.1174]), tensor([1., 1., 1.]), None)

In [129]:
#第一种方法，使用grad()获取中间变量的梯度
x = t.ones(3, requires_grad = True)
w = t.rand(3, requires_grad = True)
y = w * x
z = y.sum()
#z对y的梯度，隐式调用backward()
t.autograd.grad(z, y)

(tensor([1., 1., 1.]),)

In [131]:
#第二种方法,使用hook
#hook是一个函数，输入是梯度，不应该有返回值
def Variable_hook(grad):
    print('y的梯度：{}'.format(grad))
    
x = t.ones(3, requires_grad = True)
w = t.rand(3, requires_grad = True)
y = w * x
#注册hook
hook_handle = y.register_hook(Variable_hook)
z = y.sum()
z.backward()

#除非你每次都要用hook，否则用完后记得移除hook
hook_handle.remove()

y的梯度：tensor([1., 1., 1.])


最后再来看看variable中grad属性和backward函数`grad_variables`参数的含义，这里直接下结论：

- variable $\textbf{x}$的梯度是目标函数${f(x)} $对$\textbf{x}$的梯度，$\frac{df(x)}{dx} = (\frac {df(x)}{dx_0},\frac {df(x)}{dx_1},...,\frac {df(x)}{dx_N})$，形状和$\textbf{x}$一致。
- 对于y.backward(grad_variables)中的grad_variables相当于链式求导法则中的$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \frac{\partial y}{\partial x}$中的$\frac{\partial z}{\partial y}$。z是目标函数，一般是一个标量，故而$\frac{\partial z}{\partial y}$的形状与variable $\textbf{y}$的形状一致。`z.backward()`在一定程度上等价于y.backward(grad_y)。`z.backward()`省略了grad_variables参数，是因为$z$是一个标量，而$\frac{\partial z}{\partial z} = 1$

In [137]:
x = t.Tensor([0, 1, 2]).requires_grad_()
y = x ** 2 + x * 2
z = y.sum()
z.backward() #由z开始反向传播
x.grad

tensor([2., 4., 6.])

In [138]:
x = t.Tensor([0, 1, 2]).requires_grad_()
y = x ** 2 + x * 2
z = y.sum()
y_grad = t.Tensor([1, 1, 1])  #dz / dy
y.backward(y_grad)
x.grad

tensor([2., 4., 6.])

In [143]:
#如果某一个variable的volatile属性被设为True，那么所有依赖它的节点volatile属性都为True。volatile属性为True的节点不会求导
x = t.ones(3, 4).volatile
w = t.rand(3, 4)
y = w * x
y.requires_grad

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: volatile was removed (Variable.volatile is always False)
  


False

另外值得注意的是，只有对variable的操作才能使用autograd，如果对variable的data直接进行操作，将无法使用反向传播。除了对参数初始化，一般我们不会修改variable.data的值。

在PyTorch中计算图的特点可总结如下：

- autograd根据用户对variable的操作构建其计算图。对变量的操作抽象为`Function`。
- 对于那些不是任何函数(Function)的输出，由用户创建的节点称为叶子节点，叶子节点的`grad_fn`为None。叶子节点中需要求导的variable，具有`AccumulateGrad`标识，因其梯度是累加的。
- variable默认是不需要求导的，即`requires_grad`属性默认为False，如果某一个节点requires_grad被设置为True，那么所有依赖它的节点`requires_grad`都为True。
- variable的`volatile`属性默认为False，如果某一个variable的`volatile`属性被设为True，那么所有依赖它的节点`volatile`属性都为True。volatile属性为True的节点不会求导，volatile的优先级比`requires_grad`高。
- 多次反向传播时，梯度是累加的。反向传播的中间缓存会被清空，为进行多次反向传播需指定`retain_graph`=True来保存这些缓存。
- 非叶子节点的梯度计算完之后即被清空，可以使用`autograd.grad`或`hook`技术获取非叶子节点的值。
- variable的grad与data形状一致，应避免直接修改variable.data，因为对data的直接操作无法利用autograd进行反向传播
- 反向传播函数`backward`的参数`grad_variables`可以看成链式求导的中间结果，如果是标量，可以省略，默认为1
- PyTorch采用动态图设计，可以很方便地查看中间层的输出，动态的设计计算图结构。

这些知识不懂大多数情况下也不会影响对pytorch的使用，但是掌握这些知识有助于更好的理解pytorch，并有效的避开很多陷阱